In [4]:
import random
import midiutil
import numpy as np
from collections import Counter
import random

np.random.seed(42)

# 生成音乐文件
def makefile(music):
    midi_file = midiutil.MIDIFile(1)
    track = 0
    time = 0
    channel = 0
    velocity = 100
    midi_file.addTrackName(track, time, "Sample Track")
    midi_file.addTempo(track, time, 240)
    for (note, duration) in music:
        midi_file.addNote(track, channel, note, time, duration, velocity)
        time += duration
    with open("output.mid", "wb") as output_file:
        midi_file.writeFile(output_file)

# 生成初始音乐文件
def make_initial_file(lst):
    midi_file = midiutil.MIDIFile(1)
    track = 0
    time = 0
    channel = 0
    velocity = 100
    midi_file.addTrackName(track, time, "Sample Track")
    midi_file.addTempo(track, time, 240)
    for note in lst:
        midi_file.addNote(track, channel, note, time, 1, velocity)
        time += 1
    with open("initial.mid", "wb") as output_file:
        midi_file.writeFile(output_file)

# 计算转移矩阵
def calculate(sequence):
    matrix = np.zeros([27, 27])
    for i in range(len(sequence)):
        if i==0:
            continue
        else:
            matrix[sequence[i-1]-53][sequence[i]-53] += 1
    for i in range(len(matrix)):
        if np.sum(matrix[i]) == 0:
            matrix[i] = np.ones(27) / 27
        else:
            matrix[i] = matrix[i] / np.sum(matrix[i])
    return matrix

# 给定一段初始音乐
# initial_list = [66, 73, 66, 78, 66, 73, 66, 66, 73, 66, 78, 66, 73, 66, 66, 66]
initial_list = [63, 63, 63, 63, 63, 58, 58, 56, 58, 60, 60, 63, 63, 63, 63, 63, 68, 67, 63, 67, 68, 63, 68, 70, 72, 63, 70, 60, 68, 67, 65, 67, 68, 63, 56, 56, 65, 63, 56, 56, 65, 67]

# 生成初始音乐文件
make_initial_file(initial_list)

# 计算转移矩阵
transition_probabilities = (calculate(initial_list))

# 音符时值转换
def random2duration(random_numuber):
    # {'quaver':八分音符, 'crotchet':四分音符, 'minim':二分音符, 'semibreve':全音符, 'triplet':三连音}
    number2duration_dict = dict({0:'quaver', 1:'crotchet', 2:'minim', 3:'semibreve', 4:'triplet'})
    duration2time_dict = dict({'quaver':0.5, 'crotchet':1, 'minim':2, 'semibreve':4, 'triplet':2/3})
    return duration2time_dict[number2duration_dict[random_numuber]]

# 定义音乐片段编码方式
def create_music(start_note, time_length=16):
    note_list = np.arange(53,80).tolist()
    notes = [start_note]
    duration = np.array([])
    while np.sum(duration) < time_length:                       # 先把每个音符的时值定下来，然后按照时值填充音高
        if duration.shape[0] > 0:
            if abs(duration[-1] - 2/3) < 1e-5:                  # 补齐成三连音
                duration = np.append(duration, 2/3)
                duration = np.append(duration, 2/3)
        duration = np.append(duration, random2duration(random.choices([0, 1, 2, 3, 4], weights=[0.2, 0.3, 0.3, 0.1, 0.1])[0]))
    duration[-1] = time_length - np.sum(duration[:-1])          # 最后一个音的时值要改为 16-(前面所有音的时值之和)
    for i in range(duration.shape[0]):
        notes.append(random.choices(note_list, weights=transition_probabilities[notes[-1] - 53])[0])
    return list(zip(notes, duration))

# 设计适应度函数
def fitness(music):
    notes, duration = zip(*music)
    # 参数选定
    alpha, beta, gamma = 5, 1, 3
    # 计算音高的适应度，4-大三度，5-纯四度，7-纯五度，12-纯八度
    notes_score = []
    notes_dict = dict({0:1, 1:-1, 2:-1, 3:1, 4:2, 5:2, 6:0, 7:2, 8:0, 9:1, 10:-1, 11:-1, 12:1})
    for i in range(len(notes)-1):
        if abs(notes[i+1] - notes[i]) > 12:
            notes_score.append(-2)
        else:
            notes_score.append(notes_dict[abs(notes[i+1] - notes[i])])
    # 计算节奏的适应度
    time_length = sum(duration)
    duration_score = (time_length - abs(len(duration) - time_length/2))/time_length
    for i in range(len(duration)):
        if duration[i] - 2/3 < 1e-5:
            if i + 2 < len(duration) and duration[i + 1] - 2/3 < 1e-5 and duration[i + 2] - 2/3 < 1e-5:
                duration_score += 0.5
                if notes[i] == notes[i + 1] and notes[i + 1] == notes[i + 2]:
                    duration_score += 0.5
            break
    for i in range (1, len(duration)):
        if np.sum(np.array(duration[:i])) % 4 == 0:
            duration_score += 4/len(duration)
    score = alpha * np.array(notes_score).mean() + beta * np.exp(-np.array(notes_score).var()) + gamma * duration_score
    return score

# 初始化种群
def init_population(start_note, pop_size, time_length):
    population = []
    for i in range(pop_size):
        population.append(create_music(start_note, time_length))
    return population

# 交叉操作
def crossover(ind1, ind2):
    length = min(len(ind1)-2, len(ind2)-2)
    pos = random.randint(1, length)
    return ind1[:pos]+ind2[pos:], ind2[:pos]+ind1[pos:]

# 变异操作
def mutation(individual, pmut):
    notes, duration = zip(*individual)
    notes = list(notes)
    note_list = np.arange(53,80).tolist()
    for i in range(1, len(notes)):
        if random.random() < pmut:
            if notes[i-1] in note_list: 
                notes[i] = random.choices(note_list, weights=transition_probabilities[notes[i-1] - 53])[0]
            else:
                notes[i] = random.choices(note_list)[0]
    # 对时值进行变异
    duraiton = list(duration)
    time_length = sum(duration)
    i = 0
    new_duration = []
    new_notes = []
    while i < len(duration):
        if random.random() < pmut: # 变异
            # 遇到三连音
            if duration[i] == 2/3:
                tt = random2duration(random.choices([0, 1, 2, 3, 4], weights=[0.2, 0.3, 0.3, 0.1, 0.1])[0])
                if tt == 2/3: # 还是三连音 不变
                    for k in range(3):
                        new_duration.append(2/3)
                        new_notes.append(notes[i])
                else:
                    new_notes.append(notes[i])
                    if sum(new_duration) + tt >= time_length:
                        new_duration.append(time_length - sum(new_duration))
                        break
                    else:
                        new_duration.append(tt)
                i += 2
            # 其他情况
            else:
                if i == len(duration) - 1: # 最后一个音符
                    if time_length - sum(new_duration) <= 4:
                        new_notes.append(notes[i])
                        new_duration.append(time_length - sum(new_duration))
                    else:
                        while time_length - sum(new_duration) > 4: # 为了方便 这边不用三连音 可以在下次变异中出现
                            new_duration.append(random2duration(random.choices([0, 1, 2, 3], weights=[0.2, 0.4, 0.3, 0.1])[0]))
                            if new_notes[-1] in note_list:
                                new_notes.append(random.choices(note_list, weights=transition_probabilities[new_notes[-1] - 53])[0])
                            else:
                                new_notes.append(random.choices(note_list)[0])
                        new_duration.append(time_length - sum(new_duration))
                        if new_notes[-1] in note_list:
                                new_notes.append(random.choices(note_list, weights=transition_probabilities[new_notes[-1] - 53])[0])
                        else:
                            new_notes.append(random.choices(note_list)[0])
                else: # 不是最后一个音符
                    tt = random2duration(random.choices([0, 1, 2, 3, 4], weights=[0.2, 0.3, 0.3, 0.1, 0.1])[0])
                    # 如果变成了三连音
                    if tt == 2/3:
                        if sum(new_duration) + 1 > time_length:
                            new_duration.append(time_length - sum(new_duration))
                            new_notes.append(notes[i])
                            break
                        else:
                            for k in range(3):
                                new_duration.append(2/3)
                                new_notes.append(notes[i])
                            if sum(new_duration) == time_length:
                                break
                    # 没变成三连音
                    else:
                        if sum(new_duration) + tt > time_length:
                            new_duration.append(time_length - sum(new_duration))
                            new_notes.append(notes[i])
                            break
                        else:
                            new_duration.append(tt)
                            new_notes.append(notes[i])
                        
        else:
            if duration[i] == 2/3:
                for k in range(3):
                    new_duration.append(2/3)
                    new_notes.append(notes[i])
                i += 2
            else:
                new_duration.append(duration[i])
                new_notes.append(notes[i])

        i += 1

    if len(new_duration) != len(new_notes):
        print('error')
        print(new_duration)
        print(new_notes)
    return list(zip(new_notes, new_duration))

# 倒影操作
def inversion(music):
    notes, duration = zip(*music)
    return list(zip([127 - note for note in notes], duration))

# 倒序操作
def retrograde(music):
    notes, duration = zip(*music)
    return list(zip(notes[::-1], duration[::-1]))

# 移调操作
def transposition(music):
    notes, duration = zip(*music)
    n = random.randint(-6, 6)
    return list(zip([note + n for note in notes], duration))

# 生成最终结果
def generate_music(start_note, length, pop_size=20, ngen=1000, pmut=0.05):
    # 初始化种群
    population = init_population(start_note, pop_size, length)
    # 迭代过程
    for i in range(ngen):
        fitnesses = [fitness(music) for music in population]
        # 选择
        select = random.choices(population, weights=fitnesses, k=10)
        for j in range(5):
            parent1, parent2 = select[j*2], select[j*2+1]
            # 交叉、变异
            child1, child2 = crossover(parent1, parent2)
            population.append(child1)
            population.append(child2)
            population.append(mutation(parent1, pmut))
            population.append(mutation(parent2, pmut))
            # 倒影、倒序、移调
            population.append(inversion(parent1))
            population.append(inversion(parent2))
            population.append(retrograde(parent1))
            population.append(retrograde(parent2))
            population.append(transposition(parent1))
            population.append(transposition(parent2))
        
        max_size = 200
        if len(population) > max_size:
            population = sorted(population, key=lambda x: -fitness(x))
            population = population[:max_size]

        best_music = max(population, key=fitness)
        if i % 50 == 0:
            print("Generation {}, best fitness: {}".format(i+1, fitness(best_music)))

    # 生成MIDI文件
    makefile(best_music)

# 测试程序
generate_music(start_note=initial_list[0], length=48)

Generation 1, best fitness: 11.740233236108823
Generation 51, best fitness: 14.73575611898859
Generation 101, best fitness: 17.821816468859893
Generation 151, best fitness: 18.94333972150641
Generation 201, best fitness: 20.739064828627782
Generation 251, best fitness: 20.97189824614415
Generation 301, best fitness: 21.448223565458598
Generation 351, best fitness: 21.692213221900435
Generation 401, best fitness: 21.940505216708768
Generation 451, best fitness: 21.940505216708768
Generation 501, best fitness: 22.189134220257912
Generation 551, best fitness: 22.189134220257912
Generation 601, best fitness: 22.189134220257912
Generation 651, best fitness: 22.189134220257912
Generation 701, best fitness: 22.189134220257912
Generation 751, best fitness: 22.189134220257912
Generation 801, best fitness: 22.189134220257912
Generation 851, best fitness: 22.189134220257912
Generation 901, best fitness: 22.189134220257912
Generation 951, best fitness: 22.189134220257912
